In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21423800
paper_name = 'brett_rao_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/journal.pone.0017619.s003.xlsx', sheet_name='Unsorted Data')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4827 x 10


In [7]:
original_data['ORF name'] = original_data['ORF name'].astype(str)

In [8]:
# Eliminate all white spaces & capitalize
original_data['ORF name'] = clean_orf(original_data['ORF name'])

In [9]:
# Translate to ORFs 
original_data['ORF name'] = translate_sc(original_data['ORF name'], to='orf')

In [10]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF name'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Order, ORF name, Gene name, Growth at pH 2.7, Growth at pH 4.0, Growth at pH 7.0, Unnamed: 6, pHv at pH 2.7, pHv at pH 4.0, pHv at pH 7.0]
Index: []


In [11]:
original_data.set_index('ORF name', inplace=True)
original_data.index.name='orf'

In [13]:
original_data = original_data[['Growth at pH 2.7','Growth at pH 4.0','Growth at pH 7.0','pHv at pH 2.7','pHv at pH 4.0','pHv at pH 7.0']].copy()

In [14]:
original_data = original_data.apply(pd.to_numeric, axis=1, errors='coerce')

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4750, 6)

# Prepare the final dataset

In [17]:
data = original_data.copy()

In [18]:
dataset_ids = [529,530,531,59,60,61]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,529,530,531,59,60,61
data_type,value,value,value,value,value,value
orf,,,,,,
YAL002W,71.886879,86.421370,92.608908,5.270618,5.433458,6.118308
YAL004W,103.177741,96.241980,93.379834,5.353766,5.189489,6.050693
YAL005C,96.746237,98.153725,95.349976,5.285176,5.289227,6.131817
YAL007C,101.414019,97.066911,96.059716,5.253891,5.288743,6.079146
YAL008W,107.313365,102.383135,96.145374,5.284032,5.277184,6.071707


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,529,530,531,59,60,61
,data_type,value,value,value,value,value,value
gene_id,orf,,,,,,
2,YAL002W,71.886879,86.421370,92.608908,5.270618,5.433458,6.118308
1863,YAL004W,103.177741,96.241980,93.379834,5.353766,5.189489,6.050693
4,YAL005C,96.746237,98.153725,95.349976,5.285176,5.289227,6.131817
5,YAL007C,101.414019,97.066911,96.059716,5.253891,5.288743,6.079146
6,YAL008W,107.313365,102.383135,96.145374,5.284032,5.277184,6.071707


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,529,530,531,59,60,61,529,530,531,59,60,61
,data_type,value,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,
2,YAL002W,71.886879,86.421370,92.608908,5.270618,5.433458,6.118308,-1.796609,-1.048740,-0.625472,-0.113409,0.584226,1.376541
1863,YAL004W,103.177741,96.241980,93.379834,5.353766,5.189489,6.050693,0.309493,-0.092979,-0.555658,0.244646,-0.551986,1.033818
4,YAL005C,96.746237,98.153725,95.349976,5.285176,5.289227,6.131817,-0.123394,0.093076,-0.377242,-0.050719,-0.087487,1.445016
5,YAL007C,101.414019,97.066911,96.059716,5.253891,5.288743,6.079146,0.190781,-0.012695,-0.312968,-0.185438,-0.089737,1.178039
6,YAL008W,107.313365,102.383135,96.145374,5.284032,5.277184,6.071707,0.587850,0.504690,-0.305211,-0.055642,-0.143572,1.140332


# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 21423800...


  0%|          | 0/6 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 6/6 [00:39<00:00,  6.52s/it]

Updating the data_modified_on field...
